In [1]:
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD

In [2]:
# from skimage import io
# from skimage.transform import resize

# data_dir = '/kaggle/input/rice-image-dataset-train-test-split/Rice_Image_Dataset/train'
# data = []
# labels = []

# for subdir in os.listdir(data_dir):
#     subdir_path = os.path.join(data_dir, subdir)
#     for file in os.listdir(subdir_path):
#         file_path = os.path.join(subdir_path, file)
#         img = io.imread(file_path)
#         img = resize(img, (224, 224, 3))
# #         data.append(img)
#         data.concatenate((data, img))
#         labels.append(subdir)

#         data = np.array(data)
#         labels = np.array(labels)
#         print(data.shape)
#         print(labels.shape)

In [3]:
# print(dataset.shape)
# print(label.shape)

In [4]:
import os
from sklearn.model_selection import train_test_split

# load the filenames and labels from the specified directory
data_dir = '/kaggle/input/rice-image-dataset-train-test-split/Rice_Image_Dataset/test'
filenames = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
labels = [(f.split('/')[-1].split('_')[0]) for f in filenames]

# split the filenames and labels into train and test sets
filenames_train, filenames_test, labels_train, labels_test = train_test_split(
    filenames,
    labels,
    test_size=0.2,
    random_state=42
)

In [5]:
CLASS_NUM = 5
BATCH_SIZE = 32
EPOCH_STEPS = 800     #int(4323/BATCH_SIZE)
IMAGE_SHAPE = (224, 224, 3)
IMAGE_TRAIN = "/kaggle/input/rice-image-dataset-train-test-split/Rice_Image_Dataset/train"
MODEL_NAME = 'modelGoogleResNetHybrid.h5'

In [6]:
# create an ImageDataGenerator object for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# create a generator for the training set
train_data = train_datagen.flow_from_directory(
    IMAGE_TRAIN,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

Found 60000 images belonging to 5 classes.


In [7]:
train_data

In [8]:
# # prepare data
# train_datagen = ImageDataGenerator(
#     rescale=1./255,
#     #rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
# )

# generator_main = train_datagen.flow_from_directory(
#     IMAGE_TRAIN,
#     target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )

# def my_generator(generator):
#     while True: # keras requires all generators to be infinite
#         data = next(generator)
#         x = data[0]
#         y = data[1], data[1], data[1]
#         yield x, y

# train_generator = my_generator(generator_main)


In [9]:
# # prepare data
# validate_datagen = ImageDataGenerator(
#     rescale=1./255,
#     #rotation_range=30,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True
# )

# generator_main = validate_datagen.flow_from_directory(
#     IMAGE_VALIDATE,
#     target_size=(IMAGE_SHAPE[0], IMAGE_SHAPE[1]),
#     batch_size=BATCH_SIZE,
#     class_mode='categorical'
# )

# validate_generator = my_generator(generator_main)


#  **Create Model**

inception module

In [10]:
def inception(x, filters):
    # 1x1
    path1 = Conv2D(filters=filters[0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)

    # 1x1->3x3
    path2 = Conv2D(filters=filters[1][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path2 = Conv2D(filters=filters[1][1], kernel_size=(3,3), strides=1, padding='same', activation='relu')(path2)
    
    # 1x1->5x5
    path3 = Conv2D(filters=filters[2][0], kernel_size=(1,1), strides=1, padding='same', activation='relu')(x)
    path3 = Conv2D(filters=filters[2][1], kernel_size=(5,5), strides=1, padding='same', activation='relu')(path3)

    # 3x3->1x1
    path4 = MaxPooling2D(pool_size=(3,3), strides=1, padding='same')(x)
    path4 = Conv2D(filters=filters[3], kernel_size=(1,1), strides=1, padding='same', activation='relu')(path4)

    return Concatenate(axis=-1)([path1,path2,path3,path4])

Auxilary output layer

In [11]:
def auxiliary(x, name=None):
    layer = AveragePooling2D(pool_size=(5,5), strides=3, padding='valid')(x)
    layer = Conv2D(filters=128, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer)
    layer = Flatten()(layer)
    layer = Dense(units=256, activation='relu')(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units=CLASS_NUM, activation='softmax', name=name)(layer)
    return layer

In [12]:
def googleResNet():
    layer_in = Input(shape=IMAGE_SHAPE)
    
    # stage-1
    layer1 = Conv2D(filters=64, kernel_size=(7,7), strides=2, padding='same', activation='relu')(layer_in)
    layer2 = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer1)
    layer3 = BatchNormalization()(layer2)

    # stage-2
    layer4 = Conv2D(filters=64, kernel_size=(1,1), strides=1, padding='same', activation='relu')(layer3)
    layer5 = Conv2D(filters=192, kernel_size=(3,3), strides=1, padding='same', activation='relu')(layer4)
    layer6 = BatchNormalization()(layer5)
    layer7 = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(layer6)
    #skip1

    # stage-3
    layer8 = inception(layer7, [ 64,  (96,128), (16,32), 32]) #3a
    layer9 = inception(layer8, [128, (128,192), (32,96), 64]) #3b
    skip1 = Concatenate(axis=-1)([layer7, layer9])
    layer10 = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(skip1)
    
    # stage-4
    layer11 = inception(layer10, [192,  (96,208),  (16,48),  64]) #4a
    aux1  = auxiliary(layer11, name='aux1')
    #skip2
    layer12 = inception(layer11, [160, (112,224),  (24,64),  64]) #4b
    layer13 = inception(layer12, [128, (128,256),  (24,64),  64]) #4c
    layer14 = inception(layer13, [112, (144,288),  (32,64),  64]) #4d
    aux2  = auxiliary(layer14, name='aux2')
    layer15 = inception(layer14, [256, (160,320), (32,128), 128]) #4e
    skip2 = Concatenate(axis=-1)([layer12, layer15])
    layer16 = MaxPooling2D(pool_size=(3,3), strides=2, padding='same')(skip2)
    
    # stage-5
    layer17 = inception(layer16, [256, (160,320), (32,128), 128]) #5a
    layer18 = inception(layer17, [384, (192,384), (48,128), 128]) #5b
    skip3 = Concatenate(axis=-1)([layer17, layer18])
    layer19 = AveragePooling2D(pool_size=(7,7), strides=1, padding='valid')(skip3)
    
    # stage-6
    layer20 = Flatten()(layer19)
    layer21 = Dropout(0.4)(layer20)
    layer22 = Dense(units=256, activation='linear')(layer21)
    main = Dense(units=CLASS_NUM, activation='softmax', name='GoogleResNet_Hybrid')(layer22)
    
    model = Model(inputs=layer_in, outputs=[main, aux1, aux2], name="")
    
    return model

In [13]:
model = googleResNet()

2022-12-08 14:33:06.721422: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 14:33:06.821155: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 14:33:06.821959: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 14:33:06.823828: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [14]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 56, 56, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 56, 56, 64)   256         max_pooling2d[0][0]              
______________________________________________________________________________________________

In [15]:
model.compile(loss='categorical_crossentropy', 
                  loss_weights={'GoogleResNet_Hybrid': 1.0, 'aux1': 0.3, 'aux2': 0.3},
                  optimizer = Adam(learning_rate=0.000001, name="Adam"), metrics=['accuracy'])

In [16]:
model.fit(x=train_data, y=None, batch_size=BATCH_SIZE, epochs=20, verbose='auto', 
          callbacks=None, validation_split=None, validation_data=None, 
          shuffle=False, class_weight=None, sample_weight=None, 
          initial_epoch=0, steps_per_epoch=EPOCH_STEPS)

2022-12-08 14:33:10.423203: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-12-08 14:33:15.034992: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


800/800 [==============================] - 393s 478ms/step - loss: 2.9013 - GoogleResNet_Hybrid_loss: 1.8116 - aux1_loss: 1.9372 - aux2_loss: 1.6950 - GoogleResNet_Hybrid_accuracy: 0.2345 - aux1_accuracy: 0.2244 - aux2_accuracy: 0.2303
Epoch 2/20
800/800 [==============================] - 354s 442ms/step - loss: 2.4131 - GoogleResNet_Hybrid_loss: 1.4711 - aux1_loss: 1.5946 - aux2_loss: 1.5455 - GoogleResNet_Hybrid_accuracy: 0.3569 - aux1_accuracy: 0.3007 - aux2_accuracy: 0.3058
Epoch 3/20
800/800 [==============================] - 337s 421ms/step - loss: 2.0392 - GoogleResNet_Hybrid_loss: 1.1772 - aux1_loss: 1.4439 - aux2_loss: 1.4294 - GoogleResNet_Hybrid_accuracy: 0.5190 - aux1_accuracy: 0.3809 - aux2_accuracy: 0.3983
Epoch 4/20
800/800 [==============================] - 331s 414ms/step - loss: 1.6023 - GoogleResNet_Hybrid_loss: 0.8457 - aux1_loss: 1.3025 - aux2_loss: 1.2197 - GoogleResNet_Hybrid_accuracy: 0.6684 - aux1_accuracy: 0.4512 - aux2_accuracy: 0.5118
Epoch 5/20
800/800 [===

In [17]:
# model.predict(validate_generator, batch_size=BATCH_SIZE, use_multiprocessing=True)

In [18]:
model.save(MODEL_NAME)

In [19]:
# # Import necessary packages
# from keras.models import load_model
from keras.preprocessing import image
# import numpy as np

# Load the trained model
# model = load_model('my_model.h5')

# Load the image to be tested
img1 = image.load_img('/kaggle/input/rice-image-dataset-train-test-split/Rice_Image_Dataset/test/Arborio/Arborio (730).jpg'
                     , target_size=(224, 224))

# Convert the image to a numpy array
img_array1 = image.img_to_array(img1)

# Add an extra dimension to the image (since Keras expects a 4D input)
img_array1 = np.expand_dims(img_array1, axis=0)

# Use the model to predict the class of the image
prediction1 = model.predict(img_array1)

# Print the predicted class
print(prediction1)

# Load the image to be tested
img2 = image.load_img('/kaggle/input/rice-image-dataset-train-test-split/Rice_Image_Dataset/test/Arborio/Arborio (7300).jpg'
                     , target_size=(224, 224))

# Convert the image to a numpy array
img_array2 = image.img_to_array(img2)

# Add an extra dimension to the image (since Keras expects a 4D input)
img_array2 = np.expand_dims(img_array1, axis=0)

# Use the model to predict the class of the image
prediction2 = model.predict(img_array2)

# Print the predicted class
print(prediction2)


[array([[0., 1., 0., 0., 0.]], dtype=float32), array([[0., 0., 1., 0., 0.]], dtype=float32), array([[0., 0., 1., 0., 0.]], dtype=float32)]


ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1586 predict_function  *
        return step_function(self, iterator)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1576 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1569 run_step  **
        outputs = model.predict_step(data)
    /opt/conda/lib/python3.7/si